# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_next_token_predictions(prompt_text: str, model_name: str = "gpt2", top_k: int = 10):
    """
    指定されたプロンプトに続く次のトークンの上位K個とその確率を予測します。
    また、プロンプトがどのようにトークン化されたかを表示します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。
                                     デフォルトは "gpt2"。
        top_k (int, optional): 取得する上位トークンの数。デフォルトは 10。

    Returns:
        list: 予測されたトークンとその確率のリスト。
              各要素は (トークン文字列, 確率) のタプル。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # モデルを評価モードに設定 (勾配計算をオフにするなど)
        model.eval()

        # 2. プロンプトのトークン化
        print(f"\n--- Tokenizing Prompt: \"{prompt_text}\" ---")
        # トークナイザーによっては、プロンプトの先頭にスペースが必要な場合があるため、
        # GPT-2のようなモデルではそのままで良いが、モデルに応じて調整することも考慮
        inputs = tokenizer(prompt_text, return_tensors="pt")
        input_ids = inputs["input_ids"]

        print(f"Input IDs: {input_ids.tolist()[0]}")
        tokens = [tokenizer.decode([token_id]) for token_id in input_ids[0].tolist()]
        # トークンによっては、デコード時に不要なスペースが含まれることがあるため、必要に応じて調整
        # 例: tokens = [tokenizer.convert_ids_to_tokens(token_id) for token_id in input_ids[0].tolist()]
        print(f"Tokens: {tokens}")
        print("--------------------------------------")

        # 3. 次のトークンの予測
        with torch.no_grad(): # 勾配計算を行わないコンテキスト
            outputs = model(input_ids)
            # outputs.logits の形状は (batch_size, sequence_length, vocab_size)
            # 次のトークンの予測なので、最後のトークン位置のlogitsを使用
            next_token_logits = outputs.logits[0, -1, :]

        # 4. Logitsを確率に変換 (Softmax)
        probabilities = torch.softmax(next_token_logits, dim=-1)

        # 5. 上位K個のトークンとその確率を取得
        top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k)

        results = []
        print(f"\n--- Top {top_k} next token predictions for \"{prompt_text}\" ---")
        for i in range(top_k):
            token_id = top_k_indices[i].item()
            token_probability = top_k_probabilities[i].item()
            # token_idを文字列にデコード
            # decodeメソッドはリスト形式のIDも受け付ける
            token_string = tokenizer.decode([token_id])
            results.append((token_string, token_probability))
            print(f"{i+1}. Token: \"{token_string}\" (ID: {token_id}), Probability: {token_probability:.6f}")
        print("----------------------------------------------------")
        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

if __name__ == "__main__":
    prompt = "The movie was full of"
    # GPT-2モデルを使用します。他のモデル (例: "gpt2-medium", "gpt2-large", "EleutherAI/gpt-neo-1.3B" など) も試せます。
    # 大きなモデルほど、より高品質な予測が期待できますが、計算リソースも多く必要とします。
    predicted_tokens = get_next_token_predictions(prompt, model_name="gpt2", top_k=10)

    # (オプション) 他のGPT系モデルを試す場合
    # print("\nTrying with gpt2-medium (may take longer to download/run):")
    # predicted_tokens_medium = get_next_token_predictions(prompt, model_name="gpt2-medium", top_k=10)

Loading model and tokenizer for 'gpt2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


--- Tokenizing Prompt: "The movie was full of" ---
Input IDs: [464, 3807, 373, 1336, 286]
Tokens: ['The', ' movie', ' was', ' full', ' of']
--------------------------------------

--- Top 10 next token predictions for "The movie was full of" ---
1. Token: " jokes" (ID: 14532), Probability: 0.021892
2. Token: " great" (ID: 1049), Probability: 0.018644
3. Token: " laughs" (ID: 22051), Probability: 0.011524
4. Token: " bad" (ID: 2089), Probability: 0.010874
5. Token: " surprises" (ID: 24072), Probability: 0.010667
6. Token: " references" (ID: 10288), Probability: 0.010528
7. Token: " fun" (ID: 1257), Probability: 0.009992
8. Token: " humor" (ID: 14733), Probability: 0.007415
9. Token: " "" (ID: 366), Probability: 0.007408
10. Token: " the" (ID: 262), Probability: 0.006709
----------------------------------------------------


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
